In [1]:
#TO RE-RUN
%reset -f

In [2]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from datetime import datetime
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter
import MLpipeline as MLpipeline

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### Run experiments

In [3]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "all_readmisssion_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended" # ["minimum,"extended']
    #Extended -> Subset of columns
    #Minimum -> minimum set of columns 
typeDiagnosis = "diag_1"  #["diag_1", "diag_3"]    
typeDisease = "Neoplasms_1" # ["subset","any",["Respiratory",...]]
    #subset -> Return subset of predefined disease features
    #any -> Return all disease features    
    #disase -> Return diases feature
typeDataExperiment = "disease" #["all", "disease"] 
    #all -> Include all diagnosis as columns
    #disease -> Remove diagnosis as column and keep only rows with diagnosis == 1       

In [4]:
verbose = True
cv_thr = 0.3
cv_folds = 5

tr_thrs = [1.0] # [0.2,0.4,0.6,1.0]
ts_thr = 0.30

fs_methods = ["none"] #["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["rf"] #["rf","svmRBF","logReg","knn","nn"]
lms = ["recall","f1_weighted"] #["f1_weighted","precision_weighted","roc_auc","recall]
sm_types = ["after"] #["none","after"]
sm_method = "sm_smote"

In [5]:
#Load data
df_all = MLpipeline.load_data(typeEncounter, typeDiagnosis, typeDataFeatures)
print "\nSHAPE:"
print df_all.shape
print "\nInitial columns:"
print df_all.columns

#Filter data by class
df_all = MLpipeline.filter_data_by_class(df_all, typeHypothesis)
print "\nRows by class type:"
print df_all.iloc[:,-1].sort_values().unique(), np.sum(df_all["readmitted"] == 0), np.sum(df_all["readmitted"] == 1)
    
#Get columns
colsDiseases, colsNonDiseases = MLpipeline.get_columns(df_all,typeDiagnosis)
print "\nDiseases:", colsDiseases
print "\nNon-diseases:", colsNonDiseases
    
#Load diseases
diseases = MLpipeline.get_diseases(colsDiseases, typeDisease)
print "\nTotal data:", df_all.shape
print diseases


SHAPE:
(71518, 30)

Initial columns:
Index([u'diss_1', u'adm_src_ref', u'adm_src_em', u'race_AfricanAmerican',
       u'race_Caucasian', u'race_Other', u'medSpec_cardio',
       u'medSpec_Family/GeneralPractice', u'medSpec_InternalMedicine',
       u'medSpec_surgery', u'age_cat', u'Diabetis_1', u'Circulatory_1',
       u'Digestive_1', u'Genitourinary_1', u'Poisoning_1', u'Muscoskeletal_1',
       u'Neoplasms_1', u'Respiratory_1', u'HbA1c', u'Change',
       u'time_in_hospital', u'num_lab_procedures', u'num_procedures',
       u'num_medications', u'number_outpatient', u'number_emergency',
       u'number_inpatient', u'number_diagnoses', u'readmitted'],
      dtype='object')

Rows by class type:
[0 1] 42985 28533

Diseases: [u'Diabetis_1', u'Circulatory_1', u'Digestive_1', u'Genitourinary_1', u'Poisoning_1', u'Muscoskeletal_1', u'Neoplasms_1', u'Respiratory_1']

Non-diseases: ['diss_1', 'adm_src_ref', 'adm_src_em', 'race_AfricanAmerican', 'race_Caucasian', 'race_Other', 'medSpec_cardio'

../src/MLpipeline.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_all["readmitted"][df_all["readmitted"].values > 0] = 1


In [ ]:
MLpipeline.run(df_all, colsNonDiseases, diseases, typeDataFeatures, 
               typeDataExperiment, typeEncounter, typeHypothesis, typeDiagnosis, ts_thr, tr_thrs, 
               fs_methods, sm_method, sm_types, 
               cls_methods, lms, cv_folds, cv_thr, True, False)

(2, 6)

DataSet:
**********
**********
SIZE: 1.0
DISEASE: Neoplasms_1
(2718, 22)
ALL TRAIN: (1902, 21)
TRAIN: [0's: 1327 1's: 575 ]
ALL TEST: (816, 21)
TEST: [0's: 570 1's: 246 ]

Num experiment: 0 / 1
****************
FS: none
SM: after
CLS: rf
METRIC: recall
Fitting 5 folds for each of 180 candidates, totalling 900 fits


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/si